In [1]:
import tensorflow as tf
tf.__version__

C:\Users\Johnny\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Johnny\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Johnny\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


'2.5.0'

# Step 1. importing an OpenAI Gym game

In [5]:
import gym 
import random

In [6]:
env = gym.make('Boxing-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [4]:
print(env.unwrapped.get_action_meanings())
print(len(env.unwrapped.get_action_meanings()))

['NOOP', 'FIRE', 'UP', 'RIGHT', 'LEFT', 'DOWN', 'UPRIGHT', 'UPLEFT', 'DOWNRIGHT', 'DOWNLEFT', 'UPFIRE', 'RIGHTFIRE', 'LEFTFIRE', 'DOWNFIRE', 'UPRIGHTFIRE', 'UPLEFTFIRE', 'DOWNRIGHTFIRE', 'DOWNLEFTFIRE']
18


In [5]:
print(list(range(18)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]


In [6]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice(list(range(len(env.unwrapped.get_action_meanings()))))
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-1.0
Episode:2 Score:0.0
Episode:3 Score:-1.0
Episode:4 Score:-4.0
Episode:5 Score:-1.0


# 2. Creating a network

In [8]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [9]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [15]:
# del model 
# Use this only when you face an error

In [16]:
model = build_model(height, width, channels, actions)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten (Flatten)            (None, 67584)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               34603520  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 18)                4

# Step 3 : Creating an agent and connecting the game to the network 

In [12]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from tensorflow.python.framework.ops import disable_eager_execution

In [13]:
def build_agent(model, actions):
    
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    
    return dqn

In [18]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-4))

# Step 4 : Implementing DRL model 

In [19]:
dqn.fit(env, nb_steps=10000, visualize=True, verbose=2)

Training for 10000 steps ...


C:\Users\Johnny\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


 2378/10000: episode: 1, duration: 1297.324s, episode steps: 2378, steps per second:   2, episode reward: -19.000, mean reward: -0.008 [-2.000,  2.000], mean action: 8.712 [0.000, 17.000],  loss: 26.522121, mean_q: 16.233779, mean_eps: 0.847990
 4755/10000: episode: 2, duration: 2155.581s, episode steps: 2377, steps per second:   1, episode reward:  8.000, mean reward:  0.003 [-2.000,  2.000], mean action: 8.533 [0.000, 17.000],  loss: 0.230241, mean_q: 14.932078, mean_eps: 0.679060
 7129/10000: episode: 3, duration: 2417.978s, episode steps: 2374, steps per second:   1, episode reward:  7.000, mean reward:  0.003 [-2.000,  2.000], mean action: 7.903 [0.000, 17.000],  loss: 0.199847, mean_q: 14.765865, mean_eps: 0.465265
 9516/10000: episode: 4, duration: 1917.679s, episode steps: 2387, steps per second:   1, episode reward: -1.000, mean reward: -0.000 [-2.000,  2.000], mean action: 7.722 [0.000, 17.000],  loss: 0.184591, mean_q: 14.928146, mean_eps: 0.251020
done, took 8002.704 second

In [20]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 1.000, steps: 2378
Episode 2: reward: 0.000, steps: 2379
Episode 3: reward: 1.000, steps: 2411
Episode 4: reward: -1.000, steps: 2402
Episode 5: reward: -16.000, steps: 2377
Episode 6: reward: 1.000, steps: 2411
Episode 7: reward: -4.000, steps: 2373
Episode 8: reward: -2.000, steps: 2379
Episode 9: reward: -2.000, steps: 2368
Episode 10: reward: -5.000, steps: 2379
-2.7


# Saving the weights

In [21]:
dqn.save_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

In [1]:
del model, dqn

NameError: name 'model' is not defined

In [3]:
import tensorflow as tf
import numpy as np

q = [60,41,'Jason']
print(q)

[60, 41, 'Jason']
